In [1]:
import logging
import os
import telegram
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext


In [2]:
from keras.preprocessing.image import img_to_array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [3]:
import tensorflow.keras as keras

import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
width=256
height=256
depth=3
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1

In [5]:
from sklearn.preprocessing import LabelBinarizer
import pickle

file_name2 = r"C:\Users\User\Desktop\Sem 2 2021\Projek Sarjana Muda\Bot Model\label_list.pkl"
open_file = open(file_name2, "rb")
label_list = pickle.load(open_file)
open_file.close()

label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [6]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [7]:
# Building the model.
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape)) # First convolutional layer
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same")) # Second convolutional layer
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(64, (3, 3), padding="same")) # Third convolutional layer
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same")) # Fourth convolutional layer
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(128, (3, 3), padding="same")) # Fifth convolutional layer
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(1024, kernel_regularizer='l2')) # sixth layer (Dense)
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(n_classes)) # Seventh layer (output)
model.add(Activation("softmax"))

In [8]:
#from tensorflow import keras
model.load_weights(r'C:\Users\User\Desktop\Sem 2 2021\Projek Sarjana Muda\Bot Model\AdamModel.h5')

In [9]:
import cv2
import numpy as np
from cv2 import *
default_image_size = tuple((256, 256))

In [ ]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

bot = telegram.Bot(token="1727473991:AAHaAA4OmJdFjXJGEx3SeoL7xNhHfq0IEKs")
updater = Updater(token='1727473991:AAHaAA4OmJdFjXJGEx3SeoL7xNhHfq0IEKs', use_context=True)

def sample_responses(input_text):
    user_message = str(input_text).lower()
    
    if user_message in ("hello", "hi",):
        return "Hey!"
    
    if user_message in ("how are you", "how are you?",):
        return "Im good! Thanks for asking."
    
    if user_message in ("who are you?", "who are you", "what are you?",):
        return "I am PlDD bot that will assist you in diagnosis the leaf disease!"

    if user_message in ("thank you", "thank you!", "youre good", "thanks"):
        return "My pleasure!"
    
    return "i dont understand you."

def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi! Send an image for classifying the disease.')

def photo(update, context):
    update.message.reply_text('Image received!')
    file = bot.getFile(update.message.photo[-1].file_id)
    if file:
        file.download('temp.jpg')
        
        my_picture ='temp.jpg'
        my_picture_array = convert_image_to_array(my_picture)
        my_picture_array = my_picture_array/225.0
        my_picture_array = np.expand_dims(my_picture_array, axis=0)

        update.message.reply_text('Processing image..')
        prediction = model.predict(my_picture_array)
        update.message.reply_text("The class that I predict is: " +label_binarizer.classes_[np.argmax(prediction)])
        disease = label_binarizer.classes_[np.argmax(prediction)]
        if disease == 'Black measles disease':
            update.message.reply_text('black measles or Spanish measles, has long plagued grape growers\
 with its cryptic expression of symptoms and, for a long time, a lack of identifiable causal organism(s)', parse_mode='Markdown')
            
        elif disease == 'Black rot disease':
            update.message.reply_text('Black rot, caused by the bacterium Xanthomonas campestris pv. campestris,\
 is considered the most serious disease of crucifer crops worldwide', parse_mode='Markdown')
            
        elif disease == 'Common rust disease':
            update.message.reply_text('Common corn rust, caused by the fungus Puccinia sorghi, is the most frequently \
occurring of the two primary rust diseases of corn in the U.S., but it rarely causes significant yield losses \
in Ohio field (dent) corn.', parse_mode='Markdown')
            
        elif disease == 'Early blight disease':
            update.message.reply_text('The causal pathogen of early blight is the fungus Alternaria solani. \
Symptoms of early blight occur on fruit, stem and foliage of tomatoes and stem, foliage and tubers of potatoes. \
Spots on foliage are dark brown with circular rings giving a target appearance. These spots may merge to form dead patches. \
This disease can be differentiated from late blight as white mould-bearing spores do not develop around the foliage spots \
during humid weather conditions.', parse_mode='Markdown')
            
        elif disease == 'Frogeye spot disease':
            update.message.reply_text('frogeye leaf spot are phases of a widespread and damaging disease of apple and crabapple. The fruit rot \
phase is called black rot and on the leaf it is called frogeye leaf spot. The disease can result in losses from a rotting of \
fruit before harvest and in storage, a weakening of the tree from defoliation, and a blighting and dieback of twigs and limbs \
caused by girdling cankers.', parse_mode='Markdown')
            
        elif disease == 'Gray leaf spot disease':
            update.message.reply_text('Gray leaf spot is caused by the fungus Cercospora zeae-maydis. \
Gray leaf spot is favored by wet humid weather as often found in valley microclimates. Symptoms of gray leaf \
spot are usually first noticed in the lower leaves. Initially, lesions of gray leaf spot begin as a small dot with \
a yellow halo.', parse_mode='Markdown')
            
        elif disease == 'Healthy Leaf':
            update.message.reply_text('its a healthy leaf. Take a good care of it!', parse_mode='Markdown')
            
        elif disease == 'Isariopsis leaf spot disease':
            update.message.reply_text('This disease also called Leaf blight disease that cause by pathogen \
Pseudocercospora vitis.', parse_mode='Markdown')
            
        elif disease == 'Late blight disease':
            update.message.reply_text('Late blight, also called potato blight, disease of potato and tomato plants that is caused by the water mold \
Phytophthora infestans. The disease occurs in humid regions with temperatures ranging between 4 and 29 °C (40 and 80 °F). Hot dry\
weather checks its spread. Potato or tomato plants that are infected may rot within two weeks.', parse_mode='Markdown')
            
        elif disease == 'Leaf scorch disease':
            update.message.reply_text('Leaf scorch is caused by the fungus Diplocarpon earliana. Symptoms of leaf scorch consist of numerous small, \
irregular, purplish spots or “blotches” that develop on the upper surface of leaves. The centers of the blotches become brownish. \
Blotches may coalesce until they nearly cover the leaflet, which then appears purplish to reddish to brown.', parse_mode='Markdown')
            
        elif disease == 'Non leaf':
            update.message.reply_text('Please upload a leaf picture!', parse_mode='Markdown')
            
        elif disease == 'Nothern leaf blight disease':
            update.message.reply_text('Northern corn leaf blight (NCLB), caused by the fungus Exserohilum turcicum. \
The fungus causing NCLB overwinters as mycelia and conidia on corn residues left on the soil surface. \
During warm, moist weather in early summer, new conidia are produced on the old corn residue and carried by \
the wind or rain to lower leaves of young corn plants. Infection and disease development are favored by heavy dew, \
frequent rainfall, high humidity, and moderate temperatures.', parse_mode='Markdown')
            
        elif disease == 'Rust disease':
            update.message.reply_text('Rust disease is caused by a fungal parasite that needs living plants to survive. Rust diseases occur most often \
in mild, moist conditions. Rust is spread by spores that are transferred from infected plants to healthy plants. These spores \
can be transferred either by the wind or by water, which is why rust disease often spreads after watering. Wet surfaces \
are also needed to cause infections.', parse_mode='Markdown')
            
        elif disease == 'Scab disease':
            update.message.reply_text('scab disease is caused by the fungus Venturia inaequalis. The apple scab fungus has several host-specific \
strains that can cause disease on one type of plant but not any other. For example, the strain of V. inaequalis that \
infects mountain ash will only infect other mountain ash trees and will not infect crabapple trees. Apple and crabapple \
trees are infected by the same strain of the apple scab fungus because the trees are in the same genus.', parse_mode='Markdown')
            
        else:
            update.message.reply_text('unknown disease')
    else:
        update.message.reply_text('Image could not be processed....')
    
def help_command(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text('you can use this bot to identify the plant disease by sending the picture of the leaf to me!')

def handle_message(update, context):
    text = str(update.message.text).lower()
    response = sample_responses(text)
    
    update.message.reply_text(response)

def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    """Start the bot."""
    dp = updater.dispatcher

    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))

    # on noncommand i.e message - echo the message on Telegram
    dp.add_handler(MessageHandler(Filters.text, handle_message))
    dp.add_handler(MessageHandler(Filters.photo, photo))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2021-09-08 13:19:52,246 - apscheduler.scheduler - INFO - Scheduler started
2021-09-08 14:57:13,206 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001B5FFFE15B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /bot1727473991:AAHaAA4OmJdFjXJGEx3SeoL7xNhHfq0IEKs/getUpdates
2021-09-08 14:57:13,222 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001B5FFFE1F10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /bot1727473991:AAHaAA4OmJdFjXJGEx3SeoL7xNhHfq0IEKs/getUpdates
2021-09-08 14:57:13,225 - telegram.vend

2021-09-08 14:57:18,030 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1727473991:AAHaAA4OmJdFjXJGEx3SeoL7xNhHfq0IEKs/getUpdates (Caused by NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001B60193F250>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))"
2021-09-08 14:57:21,459 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001B5FE577460>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /bot1727473991:AAHaAA4OmJdFjXJGEx3SeoL7xNhHfq0IEKs/getUpdates
2021-09-08 14:57:21,483 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING - Retrying (Retry(t

2021-09-08 14:57:45,585 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1727473991:AAHaAA4OmJdFjXJGEx3SeoL7xNhHfq0IEKs/getUpdates (Caused by NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001B6015854C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
2021-09-08 14:57:45,585 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot1727473991:AAHaAA4OmJdFjXJGEx3SeoL7xNhHfq0IEKs/getUpdates (Caused by NewConnectionError('<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x000001B6015854C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))"
2021-09-08 14:58:02,677 - telegram.vendor.ptb_urllib3.urllib3.connectionpool - WARNING -